# Equal-Weight S&P 500 Index Fund

# Setup

In [1]:
# Numerical computing.
import numpy as np

# Tabular data manipulation.
import pandas as pd

# Requests.
import requests

import xlsxwriter

import math

In [2]:
# API key.
from APIKey import APIKey

# API

**IEX Cloud is a platform that provides financial data and services.** ([API Documentation](https://iexcloud.io/docs/api/))

## Structure of an API Call

In [3]:
# We are using Sandbox option from the IEX Cloud API that is for testing and is free of cost.

# https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN} - for latest stable API version.
#'Base URL'+'Naming Convention'+'Endpoint'+'?token='+APIToken.

symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={APIKey}'
data = requests.get(api_url).json()

data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': '(GNGSEASEAKSETL)CMLDLQTB RNAA  O/',
 'calculationPrice': 'close',
 'open': 139.93,
 'openTime': 1659873533129,
 'openSource': 'alficiof',
 'close': 142.91,
 'closeTime': 1639361803136,
 'closeSource': 'alcffoii',
 'high': 142.91,
 'highTime': 1700070015928,
 'highSource': ' ecmailp rund5td1e eiye',
 'low': 139.93,
 'lowTime': 1652748048755,
 'lowSource': 'eyl  dtemdne5p1iru eaci',
 'latestPrice': 140.11,
 'latestSource': 'Close',
 'latestTime': 'July 2, 2021',
 'latestUpdate': 1626201517558,
 'latestVolume': 82292070,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 144.49,
 'delayedPriceTime': 1669748419510,
 'oddLotDelayedPrice': 144.87,
 'oddLotDelayedPriceTime': 1675178272989,
 'extendedPrice': 144.48,
 'extendedChange': -0.11,
 'extendedChangePercent': -0.00083,
 'extendedPriceTime': 1645572788245,
 'previousClose': 138.24,
 'previousVolume': 54316331,
 'change': 2.78

## Parsing Retrieved Data

In [4]:
# Latest market price, market capitalization.
data['latestPrice'], data['marketCap']

(140.11, 2367769175663)

## Adding Retrieved Data to a Pandas DataFrame

In [5]:
# Column names of required data.
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']

dataframe_test = pd.DataFrame(columns = my_columns)

dataframe_test

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [6]:
# Populate the dataframe.
dataframe_test = dataframe_test.append(
                                    pd.Series(['AAPL', 
                                              data['latestPrice'], 
                                              data['marketCap'], 
                                              'N/A'], 
                                              index = my_columns), 
                                    ignore_index = True)
dataframe_test

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,140.11,2367769175663,N/A


# Get the Data

## Import List of Stocks

In [7]:
# sp_500_stocks.csv -- list of companies in S&P 500 (ticker symbol).
stocks = pd.read_csv('../sp_500_stocks.csv')

stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 1 columns):
Ticker    505 non-null object
dtypes: object(1)
memory usage: 4.1+ KB


In [8]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Loop Through The Tickers in List of Stocks

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={APIKey}'
    
    data = requests.get(api_url).json()
    
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
    
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
Ticker                     505 non-null object
Price                      505 non-null float64
Market Capitalization      505 non-null object
Number Of Shares to Buy    505 non-null object
dtypes: float64(1), object(3)
memory usage: 15.9+ KB


In [10]:
final_dataframe.head()

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,151.45,46154658135,N/A
1,AAL,22.34,14397532106,N/A
2,AAP,211.55,13811034269,N/A
3,AAPL,145.14,2393650699656,N/A
4,ABBV,115.82,212735758883,N/A


## Batch API Calls to Improve Performance

IEX Cloud limits their batch API calls to 100 tickers per request.

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [13]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    # Creating batches.
    stocks_each_call = ','.join(symbol_groups[i])
    
    # Appending batches.
    symbol_strings.append(stocks_each_call)
    
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [14]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={APIKey}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 4 columns):
Ticker                     505 non-null object
Price                      505 non-null float64
Market Capitalization      505 non-null object
Number Of Shares to Buy    505 non-null object
dtypes: float64(1), object(3)
memory usage: 15.9+ KB


In [15]:
final_dataframe.head()

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,152.97,46314482536,N/A
1,AAL,22.38,14024493370,N/A
2,AAP,215.55,14145291511,N/A
3,AAPL,144.93,2393226229959,N/A
4,ABBV,118.79,208555315345,N/A


# Number of Shares to Buy

In [16]:
# 1 million US$.
portfolio_size = 1000000

In [17]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,152.97,46314482536,12
1,AAL,22.38,14024493370,88
2,AAP,215.55,14145291511,9
3,AAPL,144.93,2393226229959,13
4,ABBV,118.79,208555315345,16
...,...,...,...,...
500,YUM,119.91,36481441063,16
501,ZBH,166.07,35051199416,11
502,ZBRA,560.76,29048750350,3
503,ZION,54.51,8975009887,36


# Export Recommended Trades

In [18]:
# Initializing XlsxWriter Object.
writer = pd.ExcelWriter('../recommended_trades/equal_weight_S&P_500.xlsx', engine='xlsxwriter')

final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [19]:
# Column formats.
# Background color.
background_color = '#0a0a23'

# Font color.
font_color = '#ffffff'

# String.
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Price.
dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

# Integer.
integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [20]:
# Apply column formats.
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [21]:
# Save the excel file.
writer.save()